# Various Tests

This notebook collects test code that was part of the files in the autosted module

## FilenameHandler

In [ ]:
from autosted.pipeline import FilenameHandler

FilenameHandler.random_prefix_length = 6
file_handler = FilenameHandler(
    "/path/to/file", ["overview", "detail"], min_index_padding_length=3
)
print(file_handler.get_path((2, 3), ending=".h5"))
print(file_handler.get_path(ending=".h5"))

file_handler = FilenameHandler(
    "/path/to/file",
    ["pre-overview", "overview", "detail"],
    min_index_padding_length=3,
)
print(file_handler.get_path((2, 3, 4), ending=".h5", mask_levels=["pre-overview"]))

## data

In [ ]:
from autosted.data import HDF5DataReader, _hdf5_group_path

def _path_test():
    pll = ("ov", "det", "det2")
    idxes = (1, 2)
    print(_hdf5_group_path(pll, idxes))


def main():
    path = "C:/Users/david/Desktop/msr-test-files/6542d40dcd6ed1833ed868ac060f73a1.h5"
    r = HDF5DataReader(path)
    print(r[(0, 22)].measurement_settings)

_path_test()

## Coordinate grouping

In [ ]:
from autosted.utils.fov_util import group_in_bounding_boxes

points = [[0, 0, 0], [0.5, 0.5, 0.5], [1, 1, 1.5]]
size = [1, 1, 1]
print(group_in_bounding_boxes(points, size))

## Task Generation & broadcasting

In [ ]:
from autosted.taskgeneration import AcquisitionTaskGenerator
from autosted.taskgeneration.taskgeneration import broadcast_updates

def main():

    from autosted.callback_buildingblocks.regular_position_generators import (
        SpiralOffsetGenerator,
    )

    spiralGen = SpiralOffsetGenerator().withStart([0, 0]).withFOV([5, 5]).withZOffset(1)
    for _ in range(5):
        print(spiralGen.get_locations())


def ATGTest():
    from unittest.mock import MagicMock
    from autosted.callback_buildingblocks.coordinate_value_wrappers import (
        ZDCOffsetSettingsGenerator,
    )

    locMock = MagicMock(return_value=[])
    locMock.get_locations = MagicMock(return_value=[])
    og = ZDCOffsetSettingsGenerator(locMock)

    pipelineMock = MagicMock()
    atg = AcquisitionTaskGenerator(0, og)
    atg(pipelineMock)

    print(locMock.get_locations())


if __name__ == "__main__":
    test_updates = [("u1", "u2", "u3"), ["v1", "v2", "v3"]]
    print(broadcast_updates(test_updates))

    u1 = ["coords1", "coords1-1"]
    u2 = ["coords2", "coords2-1"]
    u3 = ["coords3", "coords3-1"]
    v1 = [
        "settings1",
        "settings2",
    ]
    test_updates = ((u1, u2, u3), (v1,))

    for meas_updates in broadcast_updates(test_updates):
        print(broadcast_updates(meas_updates))

## Spot Detection

In [ ]:
if __name__ == "__main__":

    import logging
    import numpy as np
    from autosted.callback_buildingblocks.coordinate_value_wrappers import ValuesToSettingsDictCallback
    from autosted.detection.spot_detection import CoordinateDetectorWrapper
    from autosted.utils.parameter_constants import (OFFSET_SCAN_PARAMETERS, PIXEL_SIZE_PARAMETERS, FOV_LENGTH_PARAMETERS)
    from autosted.data import MeasurementData
    from pprint import pprint

    logging.basicConfig(level=logging.INFO)

    img = np.zeros((1, 1, 201, 201), dtype=float)
    img[0, 0, 100, 100] = 5
    img[0, 0, 20, 50] = 5

    img_ch2 = img.copy()

    off = [0, 0, 0]
    pixel_size = [0.1, 0.1, 0.1]
    fov = np.array([0.1, 0.1, 0.1]) * 200
    settings_call = ValuesToSettingsDictCallback(
        lambda: ((off, pixel_size, fov),),
        (OFFSET_SCAN_PARAMETERS, PIXEL_SIZE_PARAMETERS, FOV_LENGTH_PARAMETERS),
    )
    measurement_settings, hardware_settings = settings_call()[0][0]

    data = MeasurementData()
    data.append(hardware_settings, measurement_settings, [img, img_ch2])
    data_call = lambda: data

    def fun(img, *other_imgs, sigma=3):
        from scipy.ndimage import gaussian_laplace
        from skimage.feature import peak_local_max

        for oi in other_imgs:
            print(oi.shape)

        return peak_local_max(
            -gaussian_laplace(img.astype(float), sigma), threshold_abs=1e-6
        )

    detector = CoordinateDetectorWrapper(
        fun, data_call, channels=(0, 1), detection_kwargs={"sigma": 3}
    )
    #
    # detector = LegacySpotPairFinder(data_call, 1, [500, 0.1], plot_detections=True, return_parameter_dict=True)
    detector.normalization_range = (0, 100)
    # detector.plot_colors = ('cyan', 'magenta')

    res = detector()
    # res = ParameterValuesRepeater(SimpleManualOffset(detector, [13,13,13]), 2, nested=False)()
    pprint(res)

### Legacy spot detection

In [ ]:
if __name__ == "__main__":

    from autosted.data import MeasurementData
    from pprint import pprint

    logging.basicConfig(level=logging.INFO)

    img = np.zeros((1, 1, 201, 201), dtype=float)
    img[0, 0, 100, 100] = 5
    img[0, 0, 20, 50] = 5

    img_ch2 = img.copy()

    off = [0, 0, 0]
    pixel_size = [0.1, 0.1, 0.1]
    fov = np.array([0.1, 0.1, 0.1]) * 200
    settings_call = ValuesToSettingsDictCallback(
        lambda: ((off, pixel_size, fov),),
        (OFFSET_SCAN_PARAMETERS, PIXEL_SIZE_PARAMETERS, FOV_LENGTH_PARAMETERS),
    )
    measurement_settings, hardware_settings = settings_call()[0][0]

    data = MeasurementData()
    data.append(hardware_settings, measurement_settings, [img, img_ch2])
    data_call = lambda: data

    detector = SimpleSingleChannelSpotDetector(
        data_call, 1, 0.1, plot_detections=True, return_parameter_dict=False
    )
    detector.invert_dimensions = (False, False, True)
    #
    # detector = LegacySpotPairFinder(data_call, 1, [500, 0.1], plot_detections=True, return_parameter_dict=True)
    detector.normalization_range = (0, 100)
    # detector.plot_colors = ('cyan', 'magenta')

    res = detector()
    # res = ParameterValuesRepeater(SimpleManualOffset(detector, [13,13,13]), 2, nested=False)()
    pprint(res)


## ROI detection

In [ ]:
if __name__ == "__main__":

    from pprint import pprint
    from autosted.callback_buildingblocks.coordinate_value_wrappers import ValuesToSettingsDictCallback
    import numpy as np
    import logging
    from skimage.measure import regionprops
    from autosted.utils.parameter_constants import (OFFSET_SCAN_PARAMETERS, PIXEL_SIZE_PARAMETERS, FOV_LENGTH_PARAMETERS)
    from autosted.detection.roi_detection import SegmentationWrapper, ROIDetectorWrapper
    from autosted.data import MeasurementData

    logging.basicConfig(level=logging.INFO)

    img = np.zeros((1, 5, 201, 201), dtype=float)

    img[0, 0, 100:111, 100:111] = 5
    img[0, 0, 20:45, 50:85] = 10

    off = [0, 0, 0]
    pixel_size = [0.1, 0.1, 0.1]
    fov = np.array([0.1, 0.1, 0.1]) * (201 - 1)

    settings_call = ValuesToSettingsDictCallback(
        lambda: ((off, pixel_size, fov),),
        (OFFSET_SCAN_PARAMETERS, PIXEL_SIZE_PARAMETERS, FOV_LENGTH_PARAMETERS),
    )
    measurement_settings, hardware_settings = settings_call()[0][0]

    data = MeasurementData()
    data.append(hardware_settings, measurement_settings, [img])
    data_call = lambda: data

    def detection_fun(img, thresh=0):
        from scipy.ndimage import label

        return label(img > thresh)[0]

    def detection_rois(img, thresh=0):
        from scipy.ndimage import label

        return [r.bbox for r in regionprops(label(img > thresh)[0])]

    callback = SegmentationWrapper(
        detection_fun,
        data_call,
        return_parameter_dict=True,
        detection_kwargs={"thresh": 0.1},
        regionprops_filters={"area": (24, 2000)},
    )
    callback = ROIDetectorWrapper(detection_rois, data_call, return_parameter_dict=True,
                                   detection_kwargs={'thresh': 1})
    callback.invert_dimensions = (False, True, True)
    callback.plot_detections = True

    res = callback()

    pprint(res)

### Legacy nucleus detection

In [ ]:
if __name__ == "__main__":

    from autosted.data import MeasurementData
    from pprint import pprint
    import logging
    from autosted.callback_buildingblocks.coordinate_value_wrappers import (
        ValuesToSettingsDictCallback,
    )
    from skimage.io import imread

    img = imread("/Users/david/Downloads/dapi_nuclei.tif")
    img = img.reshape((1, 1, img.shape[0], img.shape[1]))

    logging.basicConfig(level=logging.INFO)

    off = [0, 0, 0]
    pixel_size = [0.1, 0.1, 0.1]
    fov = np.array([0.1, 0.1, 0.1]) * np.array(img.shape[1:])
    settings_call = ValuesToSettingsDictCallback(
        lambda: ((off, pixel_size, fov),),
        (OFFSET_SCAN_PARAMETERS, PIXEL_SIZE_PARAMETERS, FOV_LENGTH_PARAMETERS),
    )

    measurement_settings, hardware_settings = settings_call()[0][0]

    data = MeasurementData()
    data.append(hardware_settings, measurement_settings, [img])
    data_call = lambda: data

    detector = SimpleNucleusMidplaneDetector(data_call, plot_detections=True)
    detector = CellposeNucleusMidplaneDetector(
        data_call, diameter=20, plot_detections=True, manual_offset=1
    )
    # res = detector()

    from autosted.callback_buildingblocks.coordinate_value_wrappers import (
        ScanFieldSettingsGenerator,
    )

    res = ScanFieldSettingsGenerator(detector, True)()
    pprint(res)

## Acceptance check

In [ ]:
if __name__ == "__main__":

    import numpy as np
    from autosted.taskgeneration import AcquisitionTaskGenerator
    from autosted.callback_buildingblocks.static_settings import (
        ScanModeSettingsGenerator,
    )
    from autosted.data import MeasurementData
    from autosted.detection.clasification_check import AcceptanceCheck

    logging.basicConfig(level=logging.INFO)

    data = MeasurementData()
    data.append({}, {}, np.zeros((1, 1, 100, 100)))
    data_call = lambda: data

    gen = AcquisitionTaskGenerator(
        "test",
        ScanModeSettingsGenerator("xy"),
        AcceptanceCheck(lambda *x: True, data_call),
    )
    _, task = gen()
    print(task[0].get_all_updates(True))

## Coordinate Wrappers

In [ ]:
from autosted.callback_buildingblocks.coordinate_value_wrappers import *

if __name__ == "__main__":

    # dummy callback returning list of 3D coordinates
    positions = [[1, 2, 3], [4, 5, 6]]
    position_callback = lambda: positions

    # test ScanOffsetsSettingsGenerator / Stage... / ZDC...
    gen = ZDCOffsetSettingsGenerator(position_callback, False)
    res = gen()
    pprint.pprint(res)

    # dummy callback returning pairs of 3D coordinates
    # -> can be interpreted as offset, size for ScanFieldSettingsGenerator
    # or as pairs of offsets from MultipleScanOffsetsSettingsGenerator
    coord_pairs = [((1, 2, 3), (1, 2, 3)), ((2, 3, 4), (5, 6, 7))]
    coord_pairs_callback = lambda: coord_pairs

    # test field/ROI settings generator
    gen = ScanFieldSettingsGenerator(coord_pairs_callback, True)
    res = gen()
    pprint.pprint(res)

    # test multiple offsets generator
    gen = MultipleScanOffsetsSettingsGenerator(coord_pairs_callback, True)
    res = gen()
    pprint.pprint(res)


## Regular position generators

In [ ]:
from autosted.callback_buildingblocks.regular_position_generators import *

def __test_main():

    logging.basicConfig(level=logging.INFO)

    positions = [[1, 2], [3, 4]]
    generator = PositionListOffsetGenerator(positions, return_parameter_dict=True)
    print(generator())

    generator = SpiralOffsetGenerator([5, 5], [2, 0, 0], return_parameter_dict=True)
    print(generator())
    print(generator())


if __name__ == "__main__":
    __test_main()

## Static settings generators

In [ ]:
from autosted.callback_buildingblocks.static_settings import *

if __name__ == "__main__":

    generator = DifferentFirstFOVSettingsGenerator(
        [None, None, None], first_lengths=[15e-6, None, None]
    )
    print(generator())
    print(generator())

    generator = PinholeSizeSettingsGenerator(25e-6)
    print(generator())